In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip install dreamsboard -U

### 介绍
使用langchain进行任务规划，构建子任务的会话场景资源，通过MCTS任务执行器，来让每个子任务通过在上下文中资源，通过自身反思探索来获取自身对问题的最优答案；这种方式依赖模型的对齐偏好，我们在每种偏好上设计了一个工程框架，来完成自我对不同答案的奖励进行采样策略


## 使用

### 构建任务

- 初始化任务引擎 StructuredTaskStepStoryboard传入需要的任务

- loader_task_step_iter_builder 构建任务的子任务，完成后SimpleTaskStepStore可获取子任务信息

- init_task_engine_dreams 初始化场景加载资源，对子任务进行规划，获取会话的资源信息

```

    os.environ["AEMO_REPRESENTATION_PROMPT_TEMPLATE"] = AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST
    os.environ["STORY_BOARD_SCENE_TEMPLATE"] = STORY_BOARD_SCENE_TEMPLATE_TEST
    os.environ["STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE"] = STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST
    os.environ["EDREAMS_EVOLUTIONARY_TEMPLATE"] = EDREAMS_EVOLUTIONARY_TEMPLATE_TEST
    os.environ["EDREAMS_PERSONALITY_TEMPLATE"] = EDREAMS_PERSONALITY_TEMPLATE_TEST
    os.environ["DREAMS_GEN_TEMPLATE"] = DREAMS_GEN_TEMPLATE_TEST
```

- init_task_engine_storyboard_executor 构建会话场景执行器，初始化一个会话

- storyboard_code_gen_builder 构建会话场景执行器, 对会话存储进行加载，加载失败重新构建

- generate_step_answer 通过会话场景 获取任务的答案





```

In [3]:
   
from dreamsboard.engine.storage.task_step_store.types import BaseTaskStepStore
from dreamsboard.engine.task_engine_builder.core import TaskEngineBuilder
from dreamsboard.engine.storage.task_step_store.simple_task_step_store import SimpleTaskStepStore
from langchain_community.chat_models import ChatOpenAI
from dreamsboard.dreams.builder_task_step.base import StructuredTaskStepStoryboard
from dreamsboard.engine.utils import concat_dirs
from dreamsboard.engine.storage.task_step_store.types import DEFAULT_PERSIST_FNAME
from dreamsboard.common.try_parse_json_object import try_parse_json_object
from dreamsboard.engine.memory.mctsr.prompt import RefineResponse
from dreamsboard.dreams.task_step_md.base import TaskStepMD
from dreamsboard.common import _get_assistants_tool

import logging
import threading
import os
from dreamsboard.dreams.task_step_to_question_chain.weaviate.prepare_load import get_query_hash
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# 控制台打印
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)

logger.addHandler(handler)

In [4]:

llm = ChatOpenAI(
    openai_api_base=os.environ.get("API_BASE"),
    model=os.environ.get("API_MODEL"),
    openai_api_key=os.environ.get("API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
kor_dreams_task_step_llm = ChatOpenAI(
    openai_api_base=os.environ.get("API_BASE"),
    model=os.environ.get("API_MODEL"),
    openai_api_key=os.environ.get("API_KEY"),
    verbose=True,
    temperature=0.95,
    top_p=0.70,
)

deepseek_llm = ChatOpenAI(
    openai_api_base=os.environ.get("DEEPSEEK_API_BASE"),
    model=os.environ.get("DEEPSEEK_API_MODEL"),
    openai_api_key=os.environ.get("DEEPSEEK_API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
zhipuai_llm = ChatOpenAI(
    openai_api_base=os.environ.get("ZHIPUAI_API_BASE"),
    model=os.environ.get("ZHIPUAI_API_MODEL"),
    openai_api_key=os.environ.get("ZHIPUAI_API_KEY"),
    verbose=True,
    temperature=0.1,
    top_p=0.9,
)
if 'glm' in os.environ.get("API_MODEL"):

    tools= [ { "type": "web_search",   "web_search": {"enable": False ,"search_result": False   }}]
else:
    tools = []
llm_with_tools = llm.bind(   tools=[_get_assistants_tool(tool) for tool in tools] )
kor_dreams_task_step_llm_with_tools = kor_dreams_task_step_llm.bind(   tools=[_get_assistants_tool(tool) for tool in tools] )

from tests.test_builder_task_step.prompts import (
    AEMO_REPRESENTATION_PROMPT_TEMPLATE as AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST,
    STORY_BOARD_SCENE_TEMPLATE as STORY_BOARD_SCENE_TEMPLATE_TEST,
    STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE as STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST,
    EDREAMS_EVOLUTIONARY_TEMPLATE as EDREAMS_EVOLUTIONARY_TEMPLATE_TEST,
    EDREAMS_PERSONALITY_TEMPLATE as EDREAMS_PERSONALITY_TEMPLATE_TEST,
    DREAMS_GEN_TEMPLATE as DREAMS_GEN_TEMPLATE_TEST,
) 
os.environ["AEMO_REPRESENTATION_PROMPT_TEMPLATE"] = AEMO_REPRESENTATION_PROMPT_TEMPLATE_TEST
os.environ["STORY_BOARD_SCENE_TEMPLATE"] = STORY_BOARD_SCENE_TEMPLATE_TEST
os.environ["STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE"] = STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE_TEST
os.environ["EDREAMS_EVOLUTIONARY_TEMPLATE"] = EDREAMS_EVOLUTIONARY_TEMPLATE_TEST
os.environ["EDREAMS_PERSONALITY_TEMPLATE"] = EDREAMS_PERSONALITY_TEMPLATE_TEST
os.environ["DREAMS_GEN_TEMPLATE"] = DREAMS_GEN_TEMPLATE_TEST


# 存储
cross_encoder_path = "/mnt/ceph/develop/jiawei/model_checkpoint/jina-reranker-v2-base-multilingual"
embed_model_path = "/mnt/ceph/develop/jiawei/model_checkpoint/m3e-base"
start_task_context = "MCTS在PRM偏好策略模型微调的应用探索综述"
builder = StructuredTaskStepStoryboard.form_builder(
    llm_runable=llm_with_tools,
    kor_dreams_task_step_llm=kor_dreams_task_step_llm_with_tools,
    start_task_context=start_task_context,
    cross_encoder_path=cross_encoder_path,
    embed_model_path=embed_model_path
)

 

/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
WARNING! top_p is not default parameter.
    

In [5]:
 
# 初始化任务引擎
task_engine_builder = builder.loader_task_step_iter_builder(allow_init=True)
while not task_engine_builder.empty():
    task_engine = task_engine_builder.get()  
    logger.info(task_engine.task_step_id)

/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
eb070cc3-ef0f-4688-b3ad-b932bb798ac1
5071151c-3339-4498-b707-450cad90a113
f9756488-9e92-41b9-98d7-cbad0b597e74
0923bad3-3bc7-437a-a6bd-0db51a28221a
66c96ccf-c4e8-4f95-adfc-965de9982c9b
14a843fc-a8a9-4135-9dbf-dacf7d517a5e
0e50671c-7df7-42a3-9f66-5ad4526ff39b


In [5]:

from dreamsboard.document_loaders.structured_storyboard_loader import StructuredStoryboard
store_load = SimpleTaskStepStore.from_persist_dir(persist_dir="./088fd419487afb7617b9fb82a8cf8b4c/storage")
 
structured_storyboard = StructuredStoryboard(json_data=[step.__dict__ for step in list(store_load.task_step_all.values())])

structured_storyboard.parse_table()

,task_step_id,shot_number,scene_number,start_task_context,aemo_representation_context,task_step_name,task_step_description,task_step_level,task_step_question,task_step_question_context,task_step_question_answer,ref_task_step_id
0,eb070cc3-ef0f-4688-b3ad-b932bb798ac1,1,story_board0,MCTS在PRM偏好策略模型微调的应用探索综述,### 1. 分析近几年研究领域的技术框架与方法论\n在计算机科学相关研究领域，对于 MCT...,分析近几年研究领域的技术框架与方法论,在计算机科学相关研究领域，对于 MCTS（蒙特卡洛树搜索）在 PRM（概率地图法）偏好策略模...,0,近几年在探索 MCTS 在 PRM 偏好策略模型微调应用时，具体是通过哪些实验方法来对比不同...,[],,
1,5071151c-3339-4498-b707-450cad90a113,2,story_board1,MCTS在PRM偏好策略模型微调的应用探索综述,### 1. 分析近几年研究领域的技术框架与方法论\n在计算机科学相关研究领域，对于 MCT...,研究论文中采用的主要框架在不同任务中的应用与变体,应用：MCTS 框架被应用于 PRM 偏好策略模型微调任务中，旨在利用 MCTS 的搜索能力...,1,在路径规划任务中，通过MCTS搜索动态调整PRM生成路径偏好时，具体是如何衡量生成的路径更符...,[],,
2,f9756488-9e92-41b9-98d7-cbad0b597e74,3,story_board2,MCTS在PRM偏好策略模型微调的应用探索综述,### 1. 分析近几年研究领域的技术框架与方法论\n在计算机科学相关研究领域，对于 MCT...,评估学术界的技术进步与局限性,技术进步：通过将 MCTS 应用于 PRM 偏好策略模型微调，在一些任务上取得了性能提升。例...,2,在将 MCTS 应用于 PRM 偏好策略模型微调取得技术进步的同时，面对模型性能提升瓶颈、大...,[],,
3,0923bad3-3bc7-437a-a6bd-0db51a28221a,4,story_board3,MCTS在PRM偏好策略模型微调的应用探索综述,### 1. 分析近几年研究领域的技术框架与方法论\n在计算机科学相关研究领域，对于 MCT...,探讨计算模型在不同数据集与应用场景下的适用性与泛化能力,适用性：在特定数据集和应用场景下，如简单环境的路径规划数据集，结合 MCTS 的 PRM 偏...,3,结合 MCTS 的 PRM 偏好策略模型在多领域、多模态数据场景下难以有效融合不同模态信息进...,[],,
4,66c96ccf-c4e8-4f95-adfc-965de9982c9b,5,story_board4,MCTS在PRM偏好策略模型微调的应用探索综述,### 1. 分析近几年研究领域的技术框架与方法论\n在计算机科学相关研究领域，对于 MCT...,分析最新算法的稳定性与容错性,稳定性：针对算法稳定性有一定优化，例如通过限制 MCTS 的搜索深度和广度，避免搜索过程中的...,4,在复杂、动态环境下，除了限制 MCTS 的搜索深度和广度外，还可以采取哪些措施来提高算法在 ...,[],,
5,14a843fc-a8a9-4135-9dbf-dacf7d517a5e,6,story_board5,MCTS在PRM偏好策略模型微调的应用探索综述,### 1. 分析近几年研究领域的技术框架与方法论\n在计算机科学相关研究领域，对于 MCT...,评估论文中提出的未来研究方向与挑战,未来研究方向：论文提出了一些新的研究方向，如探索更有效的多模态数据融合方法，以提升模型在多领...,5,论文提出的探索更有效的多模态数据融合方法这一未来研究方向，在设计通用的多模态数据融合框架时可...,[],,
6,0e50671c-7df7-42a3-9f66-5ad4526ff39b,7,story_board6,MCTS在PRM偏好策略模型微调的应用探索综述,### 1. 分析近几年研究领域的技术框架与方法论\n在计算机科学相关研究领域，对于 MCT...,研究成果、方法的创新性与应用价值,研究成果：成功探索了 MCTS 在 PRM 偏好策略模型微调中的应用，并在部分任务上取得性能...,6,MCTS在PRM偏好策略模型微调的研究成果在部分任务上取得性能提升，那么在尚未取得明显提升的...,[],,




#### 场景加载模块设计方案
```

编写符合计算机科学领域的 故事情境提示词，生成研究情境（story_scenario_context），替换现有的langchain会话模板，
1、对这个提示词所要求的输入拆分成子任务，
2、对每个子任务指令转换为子问题，召回问题前3条，
3、对召回内容与问题拼接，合成会话内容变量（scene_content）


对每个子问题相关的召回内容，转换为第一人称的会话总结（研究场景（scene_monologue_context）），

1、对召回内容与问题拼接，对每个前3条组成一个总结任务的提示词，为每个任务标记唯一编号，组成任务上下文（story_board_summary_context）
2、加载编号和story_board_summary_context，转换会话信息
```



#### MCTS设计方案

```
MCT 自优化算法代表了蒙特卡洛树搜索（MCTS）与大型语言模型的结合，将不同的场景执行任务过程抽象为搜索树结构。树上的节点代表当前不同视角下的选择策略，而边表示主体对自身的反思。该算法的操作流程遵循 MCTS 算法的一般模式。

具体来说，我们采用模型的先验知识，来让每个子任务通过在上下文中资源，通过自身反思探索来获取自身对问题的最优答案；这种方式依赖模型的对齐偏好，我们在每种偏好上设计了一个工程框架，来完成自我对不同答案的奖励进行采样策略


1、对问题生成的子任务，生成一个合理的规划的节点
2、对每个节点创建一个MCTS任务，
3、输入 problem（总问题的子任务相关的子问题）
4、评分代码重构，将片段摘录器模块集成到一个关于_evaluate_answer逻辑提示模板，模板主要作用：将每个子问题相关的loader_cosplay_builder构建一个关于evaluate_system_prompt 的生成策略，具体的为编写一个关于带有评估的评估器，由loader_cosplay_builder方法返回场景执行器（CodeGeneratorBuilder），使用add_generator添加一个问答策略(CodeGenerator)中构成问答交互，build_executor后执行  executor.chat_run() 返回_ai_message

5、自我反思代码重构,将片段摘录器模块集成到一个关于self_refine逻辑提示模板，模板主要作用：将每个子问题相关的loader_cosplay_builder构建一个关于critic_system_prompt和refine_system_prompt的生成策略，critic_system_prompt为生成一个关于子问题相关的loader_cosplay_builder中自身不完美的评价内容，refine_system_prompt为不完美评价的思考过程和评分值。
具体的为编写一个关于带有评价的生成器和反思生成器，它们由loader_cosplay_builder方法返回场景执行器（CodeGeneratorBuilder），使用add_generator添加一个问答策略(CodeGenerator)中构成问答交互，build_executor后执行  executor.chat_run() 返回_ai_message
```



#### MCTS执行中的重要环节
```
MCTS中的约束规则如下，需要保证这些节点必须符合下面所定义的基本规则

提示约束：模型在奖励评分期间必须遵守最严格的标准。生成结果需要为JSON Response format
{
    "thought": "The thought process behind the answer.",
    "answer": "A float representing the answer to the problem."
}


高分抑制：评分节点中不存在满分反馈机制；任何超过 95 分的奖励都会按固定金额减少，以遏制过高分数。

重复采样：每次访问搜索树节点都涉及对节点奖励的重复采样，以增强自我评估的可靠性。需要注意的是，当对节点的子节点进行奖励采样时，我们也会对其父节点进行奖励采样，以增加奖励采样的样本量。
```



In [6]:
 
# 初始化任务引擎
task_engine_builder = builder.loader_task_step_iter_builder(allow_init=False)
while not task_engine_builder.empty():
    task_engine = task_engine_builder.get()  
    logger.info(task_engine.task_step_id)

eb070cc3-ef0f-4688-b3ad-b932bb798ac1
5071151c-3339-4498-b707-450cad90a113
f9756488-9e92-41b9-98d7-cbad0b597e74
0923bad3-3bc7-437a-a6bd-0db51a28221a
66c96ccf-c4e8-4f95-adfc-965de9982c9b
14a843fc-a8a9-4135-9dbf-dacf7d517a5e
0e50671c-7df7-42a3-9f66-5ad4526ff39b


In [7]:
 


def worker(step: int, task_engine: TaskEngineBuilder, task_step_store: BaseTaskStepStore):
    owner = f"step:{step}, task_step_id:{task_engine.task_step_id}, thread {threading.get_native_id()}"
    logger.info(f"{owner}，任务开始")
    if not task_engine.check_engine_init():
        task_engine.init_task_engine()
        task_engine.init_task_engine_dreams()
        task_engine.init_task_engine_storyboard_executor()

    try:
        logger.info(f"{owner}，storyboard_code_gen_builder")
        code_gen_builder = task_engine.storyboard_code_gen_builder()
        task_step = task_engine.task_step_store.get_task_step(task_engine.task_step_id)
        if task_step.task_step_question_answer is None or len(task_step.task_step_question_answer) == 0:
            task_engine.generate_step_answer(code_gen_builder)
        
        logger.info(f"step:{step}, {owner}，get_mcts_node")
        mcts_node = task_engine.get_mcts_node()
        if step % 2 == 0:
            mcts_node.llm_runable = deepseek_llm
        if step % 3 == 0:
            mcts_node.llm_runable = zhipuai_llm
        logger.info(f"step:{step}, {owner}，get_mcts_node run")
        answer = mcts_node.run()
        
        mcts_node.print()
        print(answer)
        task_step.task_step_question_answer = answer 
        task_step_id = task_engine.task_step_id
        
        task_engine.task_step_store.add_task_step([task_step])
        task_step_store_path = concat_dirs(dirname=f"{builder.base_path}/storage/{task_step_id}", basename=DEFAULT_PERSIST_FNAME)
        task_engine.task_step_store.persist(persist_path=task_step_store_path) 
        
        task_step_store.add_task_step([task_step])
        task_step_store_path = concat_dirs(dirname=f"{builder.base_path}/storage", basename=DEFAULT_PERSIST_FNAME)
        task_step_store.persist(persist_path=task_step_store_path) 

    except Exception as e:
        logger.error("场景加载失败", e)

    logger.info(f"{owner}，任务结束")




if __name__ == "__main__":
    # import torch.multiprocessing as mp

    # mp.set_start_method('spawn', force=True)
    threads = []
    
    step =0
    task_step_store = builder.task_step_store
    # 初始化任务引擎
    task_engine_builder = builder.loader_task_step_iter_builder(allow_init=False)
    while not task_engine_builder.empty():
       
        task_engine = task_engine_builder.get()
        step += 1
        t = threading.Thread(target=worker,
                             kwargs={"step": step, "task_engine": task_engine, "task_step_store": task_step_store},
                             daemon=True)
        t.start()
        threads.append(t) 
    
    for t in threads:
        t.join()

step:1, task_step_id:eb070cc3-ef0f-4688-b3ad-b932bb798ac1, thread 2767566，任务开始
step:2, task_step_id:5071151c-3339-4498-b707-450cad90a113, thread 2767567，任务开始
step:3, task_step_id:f9756488-9e92-41b9-98d7-cbad0b597e74, thread 2767568，任务开始
step:4, task_step_id:0923bad3-3bc7-437a-a6bd-0db51a28221a, thread 2767569，任务开始
step:5, task_step_id:66c96ccf-c4e8-4f95-adfc-965de9982c9b, thread 2767570，任务开始
step:6, task_step_id:14a843fc-a8a9-4135-9dbf-dacf7d517a5e, thread 2767571，任务开始
step:7, task_step_id:0e50671c-7df7-42a3-9f66-5ad4526ff39b, thread 2767572，任务开始
Cache hit file_path: /tmp/query_cache/ff1b3f260e6b318a8227cf5f88ace226.json for query: 在复杂、动态环境下，除了限制 MCTS 的搜索深度和广度外，还可以采取哪些措施来提高算法在 PRM 偏好策略微调时的稳定性？ 
Using cached data for query: 在复杂、动态环境下，除了限制 MCTS 的搜索深度和广度外，还可以采取哪些措施来提高算法在 PRM 偏好策略微调时的稳定性？ 
owner:register_event thread 2767570
Cache hit file_path: /tmp/query_cache/0c26ffd0eaf298d8ead91915e8b966c2.json for query: 论文提出的探索更有效的多模态数据融合方法这一未来研究方向，在设计通用的多模态数据融合框架时可能会面临哪些具体的技术难题 ？  
owner:getlock th

Iteration complete for resource_collection_66c96ccf-c4e8-4f95-adfc-965de9982c9b.


  File "/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py", line 583, in encode
    self.to(device)
  File "/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1174, in to
    return self._apply(convert)
  File "/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/torch/nn/modules/module.py", line 780, in _apply
    module._apply(fn)
  File "/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/torch/nn/modules/module.py", line 780, in _apply
    module._apply(fn)
  File "/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/torch/nn/modules/module.py", line 780, in _apply
    module._apply(fn)
  [Previous line repeated 1 more time]
  File "/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/torch/nn/modules/module.py", line 805, in _apply
    param_applied = fn(param)
  File "/mnt/ceph/develop/jiaw

Iteration complete for resource_collection_14a843fc-a8a9-4135-9dbf-dacf7d517a5e.
Iteration complete for resource_collection_5071151c-3339-4498-b707-450cad90a113.


Thread-11 (worker):
Traceback (most recent call last):
  File "/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
owner:getlock thread 2767880, resource_id:resource_collection_eb070cc3-ef0f-4688-b3ad-b932bb798ac1
owner:lock thread 2767880, resource_id:resource_collection_eb070cc3-ef0f-4688-b3ad-b932bb798ac1
thread 2767880 开始操作：('088fd419487afb7617b9fb82a8cf8b4c', 'samples')。
thread 2767880 结束操作：('088fd419487afb7617b9fb82a8cf8b4c', 'samples')。
thread 2767880 开始操作：('088fd419487afb7617b9fb82a8cf8b4c', 'samples')。插入
thread 2767880 结束操作：('088fd419487afb7617b9fb82a8cf8b4c', 'samples')。插入
Exception in thread Thread-7 (worker):
Traceback (most recent call last):
  File "/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/mnt/ceph/develop/jiawei/conda_env/dreams/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    self.run()
  File "/mnt/cep

...

Iteration complete for resource_collection_0923bad3-3bc7-437a-a6bd-0db51a28221a.


Searching papers for query: 在将 MCTS 应用于 PRM 偏好策略模型微调取得技术进步的同时，面对模型性能提升瓶颈、大规模复杂环境下搜索效率和策略调整及时性问题，以及模型偏差和对训练数据依赖等局限性，未来可以采取哪些具体措施来加以改善？ 
Found 30 papers.
Found 28 unique paper IDs.
Fetching details for paper ID: 63dcdb422c26941cf00b6543
Fetching details for paper ID: 634f6ae490e50fcafdcb6573
Fetching details for paper ID: 652379c3939a5f4082e1c106
Fetching details for paper ID: 629041a85aee126c0fb5cbbe
Fetching details for paper ID: 65ea8c1b13fb2c6cf630dad4
Searching papers for query: MCTS在PRM偏好策略模型微调的研究成果在部分任务取得性能提升，这种创新性地结合二者并改进结构形成新微调机制的方法，未来要如何进一步拓展在多领域、多模态数据及复杂动态环境下的应用价值？ 
Fetching details for paper ID: 62c3abbb5aee126c0fc96f0e
Found 30 papers.
Found 29 unique paper IDs.
Fetching details for paper ID: 63dcdb422c26941cf00b6543
Fetching details for paper ID: 656fddf8939a5f408293c87a
Fetching details for paper ID: 65406316939a5f4082647855
Fetching details for paper ID: 6449f232582c1376bb22397e
Fetching details for paper ID: 6419208e90e50fcafda9279c
Fetching details for paper ID: 62bab8e

Iteration complete for resource_collection_f9756488-9e92-41b9-98d7-cbad0b597e74.


Fetching details for paper ID: 6535d747939a5f408295c3b8
Fetching details for paper ID: 63d9d86990e50fcafd57b269
Fetching details for paper ID: 65fc055e13fb2c6cf6df250e
Fetching details for paper ID: 6467126cd68f896efaf14d16
Fetching details for paper ID: 65b078de939a5f4082b32241
Data cached for query: MCTS在PRM偏好策略模型微调的研究成果在部分任务取得性能提升，这种创新性地结合二者并改进结构形成新微调机制的方法，未来要如何进一步拓展在多领域、多模态数据及复杂动态环境下的应用价值？  at /tmp/query_cache/2beaf39ddb88307ab3c7eaa2dadd53b7.json
owner:register_event thread 2767572
owner:getlock thread 2768550, resource_id:resource_collection_0e50671c-7df7-42a3-9f66-5ad4526ff39b
owner:lock thread 2768550, resource_id:resource_collection_0e50671c-7df7-42a3-9f66-5ad4526ff39b
thread 2768550 开始操作：('088fd419487afb7617b9fb82a8cf8b4c', 'samples')。
thread 2768550 结束操作：('088fd419487afb7617b9fb82a8cf8b4c', 'samples')。
thread 2768550 开始操作：('088fd419487afb7617b9fb82a8cf8b4c', 'samples')。插入
thread 2768550 结束操作：('088fd419487afb7617b9fb82a8cf8b4c', 'samples')。插入
Traceback (most recent call last)

Iteration complete for resource_collection_0e50671c-7df7-42a3-9f66-5ad4526ff39b.


In [ ]:

store_load = SimpleTaskStepStore.from_persist_dir(persist_dir="./af417502216ec584b5cbf3a9fda1879e/storage")
task_step_md = TaskStepMD(store_load)
md_text =   task_step_md.format_md() 
from IPython.display import Markdown, display

display(Markdown(md_text.text))


StructuredTaskStepStoryboard
    
对任务进行规划，生成段落之间组成一个动态上下文，  扩写任务步骤构建MCTS任务
    输入：
    	start_task_context： 起始任务
    	llm： 模型
    	kor_dreams_task_step_llm: 任务抽取模型
    	task_step_store: 任务存储器（SimpleTaskStepStore）
    	cross_encoder_path: ranking模型路径（sentence_transformers），当前设计耦合了业务，之后会单独设计一个召回模块

    任务：

        1、对任务（AEMO_REPRESENTATION_PROMPT_TEMPLATE）按照提示词要求进行扩写，将扩写任务步骤收集 （src/dreamsboard/dreamsboard/engine/entity/task_step、src/dreamsboard/tests/test_kor/test_kor3.py）

        2、收集每个任务后存储到磁盘（src/dreamsboard/dreamsboard/engine/storage/task_step_store）

        3、对每个子任务载入会话场景，然后按照扩写任务步骤构建，MCTS任务 loader_task_step_iter_builder


	》 TaskEngineBuilder 场景加载模块
		执行会话场景资源初始化，构建MCTS任务

    根据任务步骤，构建场景加载模块，生成资源文件csv
    根据每个任务，载入StructuredDreamsStoryboard 会话场景
    按照扩写任务步骤构建MCTS任务

		输入：
			task_step_id
			task_step_store: 任务存储器（SimpleTaskStepStore）
			start_task_context： 起始任务
			llm： 模型

		任务：
			init_task_engine：
				初始化任务引擎
        》TaskStepToQuestionChain 
        	输入：
        		client： 矢量库客户端
        		llm： 模型
					invoke_task_step_to_question：1、 对开始任务进行抽取，得到任务步骤，提示词所要求的输入拆分成子任务， 
					invoke_task_step_question_context： 2、对每个子任务指令转换为子问题，召回问题前3条，对任务步骤进行抽取，得到任务步骤的上下文
					export_csv_file_path: 3、对召回内容与问题 导出csv文件

			init_task_engine_dreams
				初始化场景加载资源
					StoryBoardDreamsGenerationChain
					对每个子任务通过职业提示词，载入会话场景
						1、构建场景信息（story_scenario_context），提示词（STORY_BOARD_SCENE_TEMPLATE）
						2、对任务上下文(story_board_summary_context)，构建第一人称数据(scene_monologue_context),提示词（STORY_BOARD_SUMMARY_CONTEXT_TEMPLATE）
						3、对任务上下文(story_board_summary_context)，获取任务分析(evolutionary_step), 提示词（EDREAMS_EVOLUTIONARY_TEMPLATE）
						4、对任务分析(evolutionary_step)，分析对话预设信息（性格）， 提示词（EDREAMS_PERSONALITY_TEMPLATE）
						5、对任务上下文(story_board_summary_context)，场景信息story_scenario_context, 第一人称数据(scene_monologue_context)，
						生成关于人物职业的引导话术，提示词（DREAMS_GEN_TEMPLATE）

			init_task_engine_storyboard_executor

				构建会话场景执行器 StructuredDreamsStoryboard
					对剧本和分析结果进行结构化，将开放问题与性格分析结果进行结合。生成情景扮演会话场景执行器
			    此过程如下
			        对开放问题结果进行抽取，得到问题内容
			        对性格分析结果进行抽取，得到性格分析结果
			        增加系统提示词
			        根据剧本与任务性格基础情景扮演代码，根据每步的抽取析得到的问题，生成问题的答案
			       	在上下文中增加，关于人物职业的引导话术
	        		导出会话场景执行器
					
	    storyboard_code_gen_builder
	    	构建会话场景执行器

	    generate_step_answer
	    	获取主进程子任务的答案
	    
	    get_mcts_node
	    	构建MCTS树, 初始化当前任务相关的MCTS节点，并返回MCTS执行器
